In [4]:
# Step 2: Import required packages
import oracledb
import pandas as pd


In [6]:
# Load your cleaned data
df = pd.read_csv("data/bank_reviews_cleaned.csv")  # Assume >1000 reviews

In [7]:
# Step 3: Connect to Oracle Database
# Update these values to match your environment
conn = oracledb.connect(
    user="system",
    password="root",  # <-- Replace with your Oracle XE password
    dsn="localhost:1521/XEPDB1"
)
cursor = conn.cursor()


In [8]:
# Step 4: Create banks table
cursor.execute("""
BEGIN
    EXECUTE IMMEDIATE '
        CREATE TABLE banks (
            bank_id NUMBER GENERATED BY DEFAULT ON NULL AS IDENTITY PRIMARY KEY,
            bank_name VARCHAR2(100) UNIQUE NOT NULL
        )';
EXCEPTION
    WHEN OTHERS THEN
        IF SQLCODE != -955 THEN RAISE; END IF;
END;
""")


In [10]:
# Step 5: Create reviews table
cursor.execute("""
BEGIN
    EXECUTE IMMEDIATE '
        CREATE TABLE reviews (
            review_id         NUMBER PRIMARY KEY,
            bank_id           NUMBER REFERENCES banks(bank_id),
            rating            NUMBER(1),
            original_text     CLOB,
            cleaned_text      CLOB,
            sentiment_label   VARCHAR2(10),
            sentiment_score   NUMBER(1),
            theme_cluster     NUMBER,
            identified_theme  VARCHAR2(200)
        )';
EXCEPTION
    WHEN OTHERS THEN
        IF SQLCODE != -955 THEN RAISE; END IF;
END;
""")


In [6]:
# Step 6: Example cleaned review data for your banks
df = pd.DataFrame({
    'review_id': [1, 2, 3],
    'bank': ['Commercial Bank of Ethiopia', 'Bank of Abyssinia', 'Dashen Bank'],
    'rating': [2, 4, 5],
    'review_text': [
        "Poor customer service and app crashes frequently.",
        "Good ATM network but slow online support.",
        "Excellent experience with fast loan approvals."
    ],
    'cleaned_text': [
        "poor customer service app crashes frequently",
        "good atm network slow online support",
        "excellent experience fast loan approvals"
    ],
    'sentiment_label': ['NEGATIVE', 'NEUTRAL', 'POSITIVE'],
    'sentiment_score': [-1, 0, 1],
    'theme_cluster': [1, 2, 0],
    'identified_theme': [['service', 'app'], ['atm', 'support'], ['loan', 'approval']]
})
df


,review_id,bank,rating,review_text,cleaned_text,sentiment_label,sentiment_score,theme_cluster,identified_theme
0,1,Commercial Bank of Ethiopia,2,Poor customer service and app crashes frequently.,poor customer service app crashes frequently,NEGATIVE,-1,1,"[service, app]"
1,2,Bank of Abyssinia,4,Good ATM network but slow online support.,good atm network slow online support,NEUTRAL,0,2,"[atm, support]"
2,3,Dashen Bank,5,Excellent experience with fast loan approvals.,excellent experience fast loan approvals,POSITIVE,1,0,"[loan, approval]"


In [7]:
# Step 7: Insert banks into the banks table
bank_name_to_id = {}
for bank in df['bank'].unique():
    bank_id_var = cursor.var(int)
    cursor.execute(
        "INSERT INTO banks (bank_name) VALUES (:1) RETURNING bank_id INTO :2",
        [bank, bank_id_var]
    )
    bank_name_to_id[bank] = bank_id_var.getvalue()[0]

bank_name_to_id


{'Commercial Bank of Ethiopia': 1, 'Bank of Abyssinia': 2, 'Dashen Bank': 3}

In [8]:
# Step 8: Insert reviews into the reviews table
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO reviews (
            review_id, bank_id, rating, original_text, cleaned_text,
            sentiment_label, sentiment_score, theme_cluster, identified_theme
        ) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9)
    """, (
        int(row['review_id']),
        bank_name_to_id[row['bank']],
        int(row['rating']),
        row['review_text'],
        row['cleaned_text'],
        row['sentiment_label'],
        int(row['sentiment_score']),
        int(row['theme_cluster']),
        ', '.join(row['identified_theme']) if isinstance(row['identified_theme'], list) else row['identified_theme']
    ))

conn.commit()
print("✅ Data inserted successfully.")


✅ Data inserted successfully.


In [9]:
# Step 9: Query the database to verify data insertion
cursor.execute("""
SELECT r.review_id, b.bank_name, r.rating, r.sentiment_label
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id
""")
results = cursor.fetchall()
results


[(1, 'Commercial Bank of Ethiopia', 2, 'NEGATIVE'),
 (2, 'Bank of Abyssinia', 4, 'NEUTRAL'),
 (3, 'Dashen Bank', 5, 'POSITIVE')]

In [10]:
# Step 10: Close connection
cursor.close()
conn.close()
